# La moneda perfecta

In [167]:
from pyquil import get_qc, Program
from pyquil.gates import H, MEASURE
from pyquil.quil import address_qubits
from pyquil.quilbase import Declare

## Declarativa

In [54]:
prog = Program(
    Declare("ro", "BIT", 1),
    H(0),
    MEASURE(0, ("ro", 0)),        
        
).wrap_in_numshots_loop(1)  

qvm = get_qc('9q-square-qvm')
result = qvm.run(qvm.compile(prog))
measurements = result.readout_data['ro']

if measurements == 0:
    print("CRUZ")
else:
    print("CARA")

CARA


## Funcional

In [55]:
def declare_memory(program, register_name, register_type, size):

    program += Declare(register_name, register_type, size)
    return program

In [56]:
def measure_qubit(program, qubit, register_name, register_position):

    program += MEASURE(qubit, (register_name, register_position))
    return program

In [57]:
def create_quantum_coin():
    
    prog = Program()    
    prog = declare_memory(prog, "ro", "BIT", 1)
    prog += H(0)
    prog = measure_qubit(prog, 0, "ro", 0)
    
    return prog

def interpret_result(bit_value):
    """Interpreta el resultado como cara o cruz"""
    if bit_value == 0:
        return "Cara"
    else:
        return "Cruz"

In [144]:
program = create_quantum_coin()

qvm = get_qc('9q-square-qvm')
result = qvm.run(program)

bit_result = result.readout_data['ro']
coin_result = interpret_result(bit_result)

print(f"Resultado: {bit_result}")
print(f"Moneda: {coin_result}")

Resultado: [[0]]
Moneda: Cara


## Ejercicio 2

Supongamos ahora que tenemos una competición entre dos usuarios A (gana cara) y B (gana cruz). 
En ella, van a lanzar una moneda al aire 50 veces, y nos interesa saber cuál de ellos gana. ¿Cómo 
debemos modificar el programa anterior para añadir el número de iteraciones e indicar quién gana? 
¿Cómo  iteramos  sobre  el  resultado?  ¿Qué  pasa  si  en  vez  de  modificar  el  número  de  iteraciones 
ejecuto múltiples veces el programa (qvm.run)?

In [147]:
TIRADAS = 50
program = create_quantum_coin().wrap_in_numshots_loop(TIRADAS)

qvm = get_qc('9q-square-qvm')
result = qvm.run(program)

bit_results = result.readout_data['ro'].flatten()
CARA = sum(1 for bit in bit_results if bit == 0)
CRUZ = TIRADAS - CARA

print(f"CARA: {CARA}")
print(f"CRUZ: {CRUZ}")

if CARA > CRUZ:
    print(f"CARA GANADOR! CON {CARA} VECES ACERTADAS")
elif CARA < CRUZ:
    print(f"CRUZ GANADOR CON {CRUZ} VECES ACERTADAS")
else:
    print("EMPATE!")


CARA: 30
CRUZ: 20
CARA GANADOR! CON 30 VECES ACERTADAS


In [152]:
num_ejecuciones = 50
resultados = []

for _ in range(num_ejecuciones):
    result = qvm.run(program)
    bit_result = result.readout_data['ro'][0][0]
    resultados.append(bit_result)

CARA = sum(1 for bit in resultados if bit == 0)
CRUZ = num_ejecuciones - CARA

print(f"CARA: {CARA}")
print(f"CRUZ: {CRUZ}")

if CARA > CRUZ:
    print(f"CARA GANADOR! CON {CARA} VECES ACERTADAS")
elif CARA < CRUZ:
    print(f"CRUZ GANADOR CON {CRUZ} VECES ACERTADAS")
else:
    print("EMPATE!")

CARA: 28
CRUZ: 22
CARA GANADOR! CON 28 VECES ACERTADAS


## Ejercicio 3
Ahora, vamos a extender el programa, participando los mismos dos competidores, pero con cuatro 
monedas distintas, lanzando las monedas 50 veces. Nos interesa conocer las medidas de todas las 
monedas para declarar al ganador. Para ello usaremos dos técnicas distintas, primero modificando el 
registro  para  almacenar  los  cuatro  valores  por  separado,  y  posteriormente  usando  la  función 
measure_all. ¿Cómo modificarías el programa en ambos casos?

### Declarativo

In [160]:
from dataclasses import dataclass

In [163]:
@dataclass
class Monedas:
    """Almacena la cantidad de caras que salen en n tiradas"""
    moneda1: int = 0
    moneda2: int = 0 
    moneda3: int = 0
    moneda4: int = 0

In [164]:
prog = Program(
    Declare("ro", "BIT", 4),
    H(0),
    H(1),
    H(2),
    H(3),
    MEASURE(0, ("ro", 0)),
    MEASURE(1, ("ro", 1)),
    MEASURE(2, ("ro", 2)),
    MEASURE(3, ("ro", 3)),        
        
).wrap_in_numshots_loop(50)  

qvm = get_qc('9q-square-qvm')
result = qvm.run(qvm.compile(prog))
measurements = result.readout_data['ro']
monedas = Monedas(
    moneda1=sum(measurements[:, 0] == 0),
    moneda2=sum(measurements[:, 1] == 0),
    moneda3=sum(measurements[:, 2] == 0),
    moneda4=sum(measurements[:, 3] == 0),
)
print(measurements)

[[0 0 1 1]
 [1 0 0 0]
 [1 1 0 0]
 [1 1 1 0]
 [0 1 0 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 1 0 0]
 [1 1 0 1]
 [0 1 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [1 0 1 1]
 [1 0 1 0]
 [0 1 0 1]
 [0 0 0 1]
 [0 1 0 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 1]
 [0 0 1 0]
 [1 1 1 1]
 [0 1 0 1]
 [1 0 1 1]
 [1 1 0 0]
 [1 0 1 0]
 [1 1 0 0]
 [0 0 1 1]
 [0 1 0 1]
 [1 0 1 0]
 [1 1 1 0]
 [1 1 1 0]
 [0 0 1 1]
 [1 1 0 1]
 [0 1 1 1]
 [0 0 1 0]
 [1 1 0 1]
 [1 1 1 1]
 [0 1 0 1]
 [0 0 0 0]
 [0 1 1 0]
 [0 1 1 1]
 [0 0 0 0]
 [0 0 1 0]
 [1 0 1 0]
 [1 0 0 0]
 [1 1 0 1]
 [0 0 0 1]
 [1 0 1 0]]


### Resultados

In [165]:
print("=== RESULTADOS POR MONEDA ===")
print(f"Moneda 1 - Caras: {monedas.moneda1}, Cruces: {50 - monedas.moneda1}")
print(f"Moneda 2 - Caras: {monedas.moneda2}, Cruces: {50 - monedas.moneda2}")
print(f"Moneda 3 - Caras: {monedas.moneda3}, Cruces: {50 - monedas.moneda3}")
print(f"Moneda 4 - Caras: {monedas.moneda4}, Cruces: {50 - monedas.moneda4}")

# Calcular totales y ganador
total_caras = monedas.moneda1 + monedas.moneda2 + monedas.moneda3 + monedas.moneda4
total_cruces = 200 - total_caras

print(f"\n=== RESULTADO FINAL ===")
print(f"TOTAL - Caras (A): {total_caras}, Cruces (B): {total_cruces}")

if total_caras > total_cruces:
    print("🏆 ¡GANADOR: A (Caras)!")
elif total_cruces > total_caras:
    print("🏆 ¡GANADOR: B (Cruces)!")
else:
    print("🤝 ¡EMPATE!")

=== RESULTADOS POR MONEDA ===
Moneda 1 - Caras: 26, Cruces: 24
Moneda 2 - Caras: 23, Cruces: 27
Moneda 3 - Caras: 26, Cruces: 24
Moneda 4 - Caras: 27, Cruces: 23

=== RESULTADO FINAL ===
TOTAL - Caras (A): 102, Cruces (B): 98
🏆 ¡GANADOR: A (Caras)!


#### Measure all

In [ ]:
def crear_programa_cuatro_monedas():
    prog = Program()
    prog = declare_memory(prog, "ro", "BIT", 4)
    
    for qubit in range(4):
        prog += H(qubit)
    
    qubits_a_medir = [0, 1, 2, 3]
    for i, qubit in enumerate(qubits_a_medir):
        prog = measure_qubit(prog, qubit, "ro", i)
    
    return prog

In [ ]:
def medir_todos_los_qubits(programa, qubits, nombre_registro):
    """Mide todos los qubits especificados y los almacena en el registro"""
    for i, qubit in enumerate(qubits):
        programa = measure_qubit(programa, qubit, nombre_registro, i)
    return programa

In [ ]:
def competicion_cuatro_monedas():
    prog = Program()
    prog = declare_memory(prog, "ro", "BIT", 4)
    
    for qubit in range(4):
        prog += H(qubit)
    
    prog = medir_todos_los_qubits(prog, [0, 1, 2, 3], "ro")
    
    return prog.wrap_in_numshots_loop(50)

In [ ]:
programa = competicion_cuatro_monedas()
qvm = get_qc('9q-square-qvm')
resultado = qvm.run(programa)
mediciones = resultado.readout_data['ro']

In [169]:
monedas = Monedas(
    moneda1=sum(mediciones[:, 0] == 0),
    moneda2=sum(mediciones[:, 1] == 0),
    moneda3=sum(mediciones[:, 2] == 0),
    moneda4=sum(mediciones[:, 3] == 0),
)

def determinar_ganador_funcional(monedas: Monedas):
    """Determina el ganador basado en el conteo total usando dataclass"""
    total_caras = monedas.moneda1 + monedas.moneda2 + monedas.moneda3 + monedas.moneda4
    total_cruces = 200 - total_caras
    
    if total_caras > total_cruces:
        return "A (Caras)", total_caras, total_cruces
    elif total_cruces > total_caras:
        return "B (Cruces)", total_caras, total_cruces
    else:
        return "EMPATE", total_caras, total_cruces

# Mostrar resultados
ganador, total_caras, total_cruces = determinar_ganador_funcional(monedas)

print("=== ANÁLISIS FUNCIONAL CON DATACLASS ===")
print(f"Moneda 1 - Caras: {monedas.moneda1}, Cruces: {50 - monedas.moneda1}")
print(f"Moneda 2 - Caras: {monedas.moneda2}, Cruces: {50 - monedas.moneda2}")
print(f"Moneda 3 - Caras: {monedas.moneda3}, Cruces: {50 - monedas.moneda3}")
print(f"Moneda 4 - Caras: {monedas.moneda4}, Cruces: {50 - monedas.moneda4}")

print(f"\n=== RESULTADO FINAL ===")
print(f"TOTAL - Caras: {total_caras}, Cruces: {total_cruces}")
print(f"🏆 GANADOR: {ganador}")

=== ANÁLISIS FUNCIONAL CON DATACLASS ===
Moneda 1 - Caras: 20, Cruces: 30
Moneda 2 - Caras: 28, Cruces: 22
Moneda 3 - Caras: 28, Cruces: 22
Moneda 4 - Caras: 23, Cruces: 27

=== RESULTADO FINAL ===
TOTAL - Caras: 99, Cruces: 101
🏆 GANADOR: B (Cruces)
